In [3]:
from utils import *

def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values=eliminate(values)
        # Your code here: Use the Only Choice Strategy
        values=only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def cross(a, b):
      return [s+t for s in a for t in b]
    
boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
# Element example:
# row_units[0] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
# This is the top most row.

column_units = [cross(rows, c) for c in cols]
# Element example:
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
# This is the left most column.

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
# Element example:
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
# This is the top left square.

unitlist = row_units + column_units + square_units

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    for x in values.keys():
        if len(values[x])==1:
            val=values[x]
            pl=peer_list(x)
            for peerkeys in pl:
                values[peerkeys]=values[peerkeys].replace(val,"")
    return values
    pass

def peer_list(key):
    rows='ABCDEFGHI'
    columns='123456789'
    unit_peers=[]
    keyRow=key[0]
    keyColumn=key[1]
    row_peers=cross(keyRow,columns) 
    col_peers=cross(rows,keyColumn)
    square_units=[cross (rs,cs) for rs in ('ABC','DEF','GHI') for cs in('123','456','789')]
    for x in square_units:
        if key in x:
            unit_peers=x
        else:
            continue
            
    peer_list=row_peers+col_peers+unit_peers
    peer_list=list(set(peer_list))
    peer_list.remove(key)
    return peer_list

def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    # TODO: Implement only choice strategy here
    
    for x in values.keys():
        if len(values[x])>1:
            val=values[x]
            valList=list(val)
            pl=peer_list(x)
            for eachval in valList:
                #print (eachval)
                peerval=[]
                for peerkeys in pl:
                    peerval.extend(list(values[peerkeys]))
                if eachval in peerval:
                    continue
                else:
                    values[x]=eachval
        else: 
            continue
    return values
    pass

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    if len(grid)!=81:
        print('Board Incomplete. Please try again')
    else:
        keys= [row_boxes for x in row_units for row_boxes in x] 
        inputList=[]
        for x in grid:
            if x=='.':
                x='123456789'
            inputList.append(x)
        return dict(zip(keys,inputList))
    pass

In [14]:
grid2 = '3..4..6.21.8..2.3.2....5.......76.147...9.5864.21..7...9..2315753..19..88...4..69'
values = grid_values(grid2)

In [15]:
display(values)

    3     123456789 123456789 |    4     123456789 123456789 |    6     123456789     2     
    1     123456789     8     |123456789 123456789     2     |123456789     3     123456789 
    2     123456789 123456789 |123456789 123456789     5     |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |123456789     7         6     |123456789     1         4     
    7     123456789 123456789 |123456789     9     123456789 |    5         8         6     
    4     123456789     2     |    1     123456789 123456789 |    7     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     9     123456789 |123456789     2         3     |    1         5         7     
    5         3     123456789 |123456789     1         9     |123456789 123456789     8     
    8     123456789 123456789 |123456789     4     123456789 |12345678

In [16]:
display(reduce_puzzle(values))

3 5 9 |4 8 1 |6 7 2 
1 4 8 |7 6 2 |9 3 5 
2 7 6 |9 3 5 |8 4 1 
------+------+------
9 8 5 |3 7 6 |2 1 4 
7 1 3 |2 9 4 |5 8 6 
4 6 2 |1 5 8 |7 9 3 
------+------+------
6 9 4 |8 2 3 |1 5 7 
5 3 7 |6 1 9 |4 2 8 
8 2 1 |5 4 7 |3 6 9 
